# Part 1

In [180]:
import re

with open('input.txt') as f:
    data = f.read().splitlines()

line_to_check = 2000000
sensors = {}
min_x = max_x = 0

x_coords = []

for line in data:
    # Regex for integers (possibly negative)
    nums = [int(x) for x in re.findall(r'-?\d+', line)]
    sensors[(nums[0], nums[1])] = (nums[2], nums[3])

    
    # We need to find the min and max x and y coordinates (including search area)
    radius = get_beacon_radius((nums[0], nums[1]), (nums[2], nums[3]))
    
    
    x_coords.append(nums[1] - radius)
    x_coords.append(nums[1] + radius)

min_x = min(x_coords)
max_x = max(x_coords)

def does_it_touch_line(sensor, sensor_radius, line_to_check):
    if abs(sensor[1] - line_to_check) < sensor_radius:
        return True

def get_sensor_coverage_on_line(sensor, sensor_radius, line_to_check, min_x):
    radius = abs(sensor[1] - line_to_check)
    covered_points = []
    for i in range(sensor_radius - radius +2):
        covered_points.append((sensor[0] + i - min_x, line_to_check))
        covered_points.append((sensor[0] - i - min_x, line_to_check))
    return covered_points

def get_beacon_radius(sensor, beacon):
    # Get the distance between the sensor and the beacon
    return abs(sensor[0] - beacon[0]) + abs(sensor[1] - beacon[1]) -1

position_covered = set()
for sensor, beacon in sensors.items():
    sensor_radius = get_beacon_radius(sensor, beacon)
    line = ['.' for _ in range(max_x - min_x + 1)]
    if does_it_touch_line(sensor, sensor_radius, line_to_check):
        # print(f"{sensor} touches line {line_to_check} with radius {sensor_radius} at position {sensor[0] - min_x}")
        locations = get_sensor_coverage_on_line(sensor, sensor_radius, line_to_check, min_x)
        for location in locations:
            if location[0] == beacon[0] - min_x and location[1] == beacon[1]:
                line[location[0]] = 'B'
            else:
                line[location[0]] = '#'
                position_covered.add(location)
                
print(f"Number of positions covered: {len(position_covered)}")

Number of positions covered: 4879972


# First attempt

Will burn your CPU and run out of memory.

Try to generate a full array with each sensors and the zone they're covering.

In [176]:
import re

with open('input.txt') as f:
    data = f.read().splitlines()

sensors = {}
min_x = min_y = max_x = max_y = 0
x_coords = y_coords = []

for line in data:
    # Regex for integers (possibly negative)
    nums = [int(x) for x in re.findall(r'-?\d+', line)]
    sensors[(nums[0], nums[1])] = (nums[2], nums[3])
    x_coords.append(nums[0])
    x_coords.append(nums[2])
    y_coords.append(nums[1])
    y_coords.append(nums[3])

min_x = min(x_coords)
max_x = max(x_coords)
min_y = min(y_coords)
max_y = max(y_coords)

def correct_position(position):
    return (position[0] - min_x, position[1] - min_y)

def correct_offset(sensors):
    sensor_calibrated = {}
    for sensor, beacon in sensors.items():
        sensor_calibrated[correct_position(sensor)] = correct_position(beacon)
    return sensor_calibrated

def get_circle_points(my_map, position, radius):
    circle_points = []
    for i in range(len(my_map)):
        for j in range(len(my_map[i])):
            if (abs(i - position[0])) + (abs(j - position[1])) == radius:
                circle_points.append((i, j))
    return circle_points
    

def build_map(sensors, max_x, max_y):
    # Empty map
    cave_map = [['. ' for x in range(max_x + 1)] for y in range(max_y + 1)]


    for sensor, beacon in sensors.items():
        radius = 0
        is_beacon_in_radius = False

        # for each sensor, draw a circle until it touches the beacon
        while not is_beacon_in_radius:
            radius += 1
            circle_points = get_circle_points(cave_map, sensor, radius)
            for point in circle_points:
                # If point is in cave_map array
                    cave_map[point[1]][point[0]] = 'X'
                    if point == beacon:
                        is_beacon_in_radius = True

    
    
    for sensor, beacon in sensors.items():
        cave_map[sensor[1]][sensor[0]] = 'S'
        cave_map[beacon[1]][beacon[0]] = 'B'
    
    return cave_map

def add_axis(cave_map, min_x, min_y):
    first_row = [str(x) for x in list(range(min_x, len(cave_map[0]) + min_x))]
    cave_map.insert(0, first_row)

    for i in range(len(cave_map)):
        cave_map[i].insert(0, str(i + min_y))

    return cave_map

def create_map(sensors, min_x, max_x, min_y, max_y):
    # Map correctly offsetted sensors
    sensors = correct_offset(sensors)

    # Get correct dimensions
    max_x, max_y = correct_position((max_x, max_y))

    # Build map
    cave_map = build_map(sensors, max_x, max_y)

    # add axis to map
    cave_map = add_axis(cave_map, min_x, min_y)
    
    return cave_map

def display_cave(cave_map):
    for row in cave_map:
        print('   '.join(row))


cave = create_map(sensors, min_x, max_x, min_y, max_y)
row_to_count = 10
print(f"row {row_to_count} has {cave[row_to_count+3].count('X')} X's")
display_cave(cave)

row 10 has 26 X's
-2   -2   -1   0   1   2   3   4   5   6   7   8   9   10   11   12   13   14   15   16   17   18   19   20   21   22   23   24   25
-1   X   X   X   X   X   X   X   X   X   X   X   X   X   .    .    .    .    .    X   X   X   X   X   X   X   X   X   . 
0   X   X   X   X   X   X   X   X   X   X   X   X   X   X   .    X   .    X   X   X   X   X   X   X   X   X   X   X
1   X   X   X   X   S   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X
2   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   S   X   X   X   X   X
3   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   S   X   X   X   X   X   X   X   X   X   X   X   X
4   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   S   B   X   X   X   X   X   X   X   X   X   X
5   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   X   . 
6   X   X   X   X   X   X   X   X   X   X